<a href="https://colab.research.google.com/github/20-hayato/practice/blob/master/speech_recog_original.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install SpeechRecognition
!pip install pydub
!apt-get install ffmpeg

In [ ]:
from zmq.constants import RECONNECT_IVL
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode
from os import path
from pydub import AudioSegment
import subprocess
from datetime import datetime, timedelta, timezone

RECORD_DIR = "record_data/"
RESULT_DIR = "result_data/"
JST = timezone(timedelta(hours=+9), 'JST')

def record_js(fname='record'):
  date = datetime.now(JST).strftime('%Y%m%d_%H:%M:%S')
  js = Javascript('''
    async function record() {
      let rec;
      let chanks;


      // HTML組み立て
      const div = document.createElement('div');
      const startRecord = document.createElement('button');
      startRecord.textContent = 'Rec';
      div.appendChild(startRecord);

      const stopRecord = document.createElement('button');
      stopRecord.textContent = 'Stop';
      stopRecord.style.display = 'none'
      div.appendChild(stopRecord);

      const audio = document.createElement('audio');
      div.appendChild(audio);

      document.body.appendChild(div);

      // Audioが有効になったら
      function handlerFunction(stream,resolve) {
          rec = new MediaRecorder(stream);
          // 録音が完了したら
          rec.ondataavailable = e => {
              chanks.push(e.data);
              if (rec.state == "inactive") {
                  let blob = new Blob(chanks, { type: 'audio/mpeg-3' });
                  audio.src = URL.createObjectURL(blob);
                  audio.controls = true;
                  audio.autoplay = true;
                  resolve();
              }
          }
      }

      startRecord.onclick = e => {
          startRecord.style.display = 'none'
          stopRecord.style.display = 'block'
          chanks = [];
          rec.start();
      }

      stopRecord.onclick = e => {
        startRecord.style.display = 'block'
        stopRecord.style.display = 'none'
        rec.stop();
      }

      function blobToBase64(blob) {
        return new Promise((resolve, _) => {
          const reader = new FileReader();
          reader.onloadend = () => resolve(reader.result);
          reader.readAsDataURL(blob);
        });
      }

      await new Promise((resolve) => {
        navigator.mediaDevices.getUserMedia({ audio: true })
            .then(stream => { handlerFunction(stream,resolve) })
      });
      let blob = new Blob(chanks, { type: 'audio/mpeg-3' });
      return await blobToBase64(blob);
    }
    ''')
  display(js)
  data = eval_js('record()')
  binary = b64decode(data.split(',')[1])

  filename = fname + "_" + date + '.mp3'
  filename_path = RECORD_DIR + filename
  with open(filename_path, 'wb') as f:
    f.write(binary)
  return filename

def mp3_to_wav(src_name):
  src_path = RECORD_DIR + src_name
  dst_name = src_name.split('.', 1)[0] + '.wav'
  dst_path = RECORD_DIR + dst_name
  subprocess.call(["ffmpeg", "-i", src_path, dst_path])
  return dst_name

def save_result(filename, result):
  txt_name = filename_wav.split('.', 1)[0] + '.txt'
  txt_path = RESULT_DIR + txt_name
  with open(txt_path, 'w') as f:
    f.write(result)

In [ ]:
import speech_recognition as sr
import glob

r = sr.Recognizer()

try:
  filename_mp3 = record_js()
  filename_wav = mp3_to_wav(filename_mp3)

  print("waiting for converting...")
  while glob.glob(RECORD_DIR + filename_wav, recursive=True) == []:
    pass

  audio_file = sr.AudioFile(RECORD_DIR + filename_wav)
  with audio_file as source:
      r.adjust_for_ambient_noise(source) #雑音対策
      audio = r.record(source)
  print("result: ")
  print(r.recognize_google(audio, language='ja-JP'))

  print("save result")
  save_result(filename_wav, r.recognize_google(audio, language='ja-JP'))

except Exception as err:
  print(str(err))

In [ ]:
# 消去用スクリプト
!ls ./record_data/
!rm ./record_data/*
!ls ./result_data/
!rm ./result_data/*